In [1]:
#using Pkg
#Pkg.activate("D:\\Science\\Code\\Julia\\envr")
using StaticArrays, DifferentialEquations, DynamicalSystems, JLD
E_, x_, y_ = 0..30, 0..1, 0..1
box = IntervalBox(E_, x_, y_)
using CairoMakie
using GLMakie

In [2]:
function TM(u, p, t)
    U(y) = p[8] + p[9] / ( 1.0 + exp( -50.0 * (y - p[7]) ) )
    σ(x) = 1.0 / ( 1.0 + exp( -20.0 * (x-p[6]) ) )
    du1 = (-u[1] + p[1] * log( 1.0 + exp( (p[5] * U(u[3]) * u[2] * u[1] + p[11]  ) / (p[1]) ) ) ) / p[2]
    du2 = (1.0 - u[2])/p[3] - U(u[3])*u[2]*u[1]
    du3 = (-u[3])/p[4] + p[10] * σ(u[2])
    return SA[du1, du2, du3]
end
function labels3d(idx, idy, idz)
    if idx == 1
        x = "E"
    elseif idx == 2
        x = "x"
    else
        x = "y"
    end
    if idy == 1
        y = "E"
    elseif idy == 2
        y = "x"
    else
        y = "y"
    end
    if idz == 1
        z = "E"
    elseif idz == 2
        z = "x"
    else
        z = "y"
    end
    return x,y,z
end;

In [3]:
t = 15000.0; tstep = 0.001
trange = range(0.0, t, step = tstep)
integ_set = (alg = RK4(), adaptive = false, dt = tstep)

const τ = 0.013;  const τD = 0.080;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4
const α = 1.58; const ΔU0 = 0.305;

In [4]:
path = "C:\\Users\\Alex\\Desktop\\repo\\2612\\0903\\dynamical-systems\\Tsodyks Markram\\Levanova\\3 набор параметров\\2М Карта спектра Ляпуновских показателей\\200x200";
filenameLS = "\\LSs_200x200.jld"
filenameu0s = "\\u0s_200x200.jld"
pathLS = path*filenameLS
pathu0s = path*filenameu0s

Λs = load(pathLS)["data"]
u0s = load(pathu0s)["data"];

In [5]:
I0range = range( -1.40, -1.709, length = 201 )
U0range = range( 0.3,  0.47, length = 201);

In [6]:
idx_I0, idx_U0 = 105, 1
I0_ = I0range[idx_I0]
U0_ = U0range[idx_U0]
println("I0:",I0_, "; U0:",  U0_)
println(Λs[idx_I0, idx_U0, :])

I0:-1.56068; U0:0.3
[0.005571183838012268, -0.162330285729737, -3.9854227511565297]


При параметрах I0:-1.61012; U0:0.3

Первый раз встречается второй аттрактор

Его начальные условия [5.029405248505961, 0.794984502437143, 0.4249747017367524]



Верхний порог по параметру для аттрактора A2

I0 = -1.596688055

Начальные условия

[12.930002594743051, 0.664533781581933, 0.43027490446713246]

In [66]:
I0_local = -1.5966880543
SA[10.543470627001867, 0.7682875252928977, 0.4357418827497073]

[10.543470627001867, 0.7682875252928977, 0.4357418827497073]


In [7]:
I0_local = -1.596688054247
p = SA[α, τ, τD, τy, J, xthr, ythr, U0_, ΔU0, β, I0_local]; # -1.7096211]

In [240]:
last_point_A2 = tr_A2[end]
println("last_point_A2: $last_point_A2")

last_point_A2: [16.602825043276685, 0.7518064079489519, 0.46099483222312704]


In [8]:
ttr = t*27.0

405000.0

In [9]:
ds_A2 = CoupledODEs(TM, SA[16.602825043276685, 0.7518064079489519, 0.46099483222312704], p, diffeq = integ_set)
tr_A2 = trajectory(ds_A2, 500.0, Δt = tstep; Ttr = ttr)
# fp, eigs, stable = fixedpoints(ds_A2, box)
ds_A2

3-dimensional CoupledODEs
 deterministic: true
 discrete time: false
 in-place:      false
 dynamic rule:  TM
 ODE solver:    RK4
 ODE kwargs:    (adaptive = false, dt = 0.001)
 parameters:    [1.58, 0.013, 0.08, 3.3, 3.07, 0.75, 0.4, 0.3, 0.305, 0.3, -1.596688054247]
 time:          405500.000206272
 state:         [7.659493607250311, 0.6773493706863961, 0.4209862754439841]


In [243]:
ts, tf = 400000, 500000
idx, idy, idz = 2, 1, 3
xl, yl, zl = labels3d(idx,idy, idz)

GLMakie.activate!()
f = Figure(resolution = (900, 900))
ax = Axis3(f[1, 1], xlabel = xl, ylabel = yl, zlabel = zl,
            xlabelsize = 35, ylabelsize = 35, zlabelsize = 35,
            xgridvisible = false, ygridvisible = false, zgridvisible = false)

lines!(tr_A2[ts:tf, idx], tr_A2[ts:tf, idy], tr_A2[ts:tf, idz], linewidth = 1.5, color = :blue)

scatter!(fp[1][idx], fp[1][idy], fp[1][idz], markersize = 10, color = :red)
display(f)

GLMakie.Screen(...)